In [ ]:
%matplotlib
#%matplotlib inline
import os
import csv
import fnmatch
import numpy as np
import datetime
import re
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import math

pd.options.mode.use_inf_as_na = True

In [ ]:
absoluteSize = False # True means absolute, False means relative

In [ ]:
TimeDwellOrig = 800
TimeFixation = 300

In [ ]:
def ComputeDwellTime(userKeys):
    # modify userKeys to include a column of time instead of progress pct, which is dependent on the then dwell time
    
    timeDwell = TimeDwellOrig
    nKey = -1
    for key in userKeys:
        nKey = nKey + 1
        if key[1] == 'IncreaseTimeDwell':
            if float(key[2]) == 1:
                timeDwell = timeDwell + 100
        elif key[1] == 'DecreaseTimeDwell':
            if float(key[2]) == 1:
                timeDwell = timeDwell - 100
        else:
            userKeys[nKey].append(str(float(key[2])*timeDwell))
    
    return userKeys

In [ ]:
# Create list of list (made of epochs) composed of times when user looked at NextPhrase

def UserLookedAtNewTrial(UserLooksAtKey):
    epoch = list()   
    epochList = list()
    
    nRow = -1
    firstNextPhrase = 0
    flagStart = 0
    firstTrial = 0
    
    # Find unique looks at new trial, by checking if progress pct is greater than previous one and store first look in a list
    
    progressNextPhrase = 0 
    
    for row in UserLooksAtKey:
        nRow = nRow + 1
        
        # first add data for first trial, before first next phrase is reached
        if firstTrial == 0:
            firstTrial = 1
            epochList.append([row, 0])
            #print(epochList)
            continue
            
        if row[1] == 'NextPhrase':
            
            # first eliminate an ongoing look
            if UserLooksAtKey[nRow-1][1] == 'NextPhrase':
                continue
            else:
                # normally for new looks
                
                # if it is a continued look, progressNextPhrase should not be 0
                #print('Continued ???', progressNextPhrase, UserLooksAtKey[nRow])
                
                if progressNextPhrase > float(UserLooksAtKey[nRow][2]):
                    
                    # save it as 0, if not a continued look
                    #print('progress is now 0')
                    progressNextPhrase = 0
#                 else:
#                     if progressNextPhrase == float(UserLooksAtKey[nRow][2]):
#                         print('ITS EQUAL!!!')
                    
                
            # now check if this look is going to be successfully selected:
            nextPhrase_selected = 0
            rowCurrent = nRow-1
            
            while nextPhrase_selected < 1:
                rowCurrent = rowCurrent + 1
                if UserLooksAtKey[rowCurrent][1] == 'NextPhrase':
                    if float(UserLooksAtKey[rowCurrent][2]) == 1:
                        #print(UserLooksAtKey[rowCurrent])
                        nextPhrase_selected = 1
                        rowCurrentEnd = rowCurrent
                        
                        
                        if progressNextPhrase != 0:
                            epochList[-1].insert(2, UserLooksAtKey[rowPreviousEnd])
                            epochList.append([UserLooksAtKey[rowCurrentStart], UserLooksAtKey[rowCurrentEnd]])
                        else:
                            epochList[-1].insert(2, UserLooksAtKey[nRow-1])
                            epochList.append([UserLooksAtKey[nRow], UserLooksAtKey[rowCurrentEnd]])
                    else:
                        continue
                else:
                    # if look at next phrase is stopped, store the previous look progress, in case it is continued later
                    progressNextPhrase = float(UserLooksAtKey[rowCurrent-1][2])
                    rowCurrentStart = nRow
                    rowPreviousEnd = nRow - 1
                    break
                    
                     
                    
                    
    # insert last element before quiting - 
    # keys looked at list
    KeysLookedAtFromUserKeys = [key[1] for key in UserLooksAtKey]
    
    # find key before quit key
    nBegin = 300
    foundQuitKey = 0
    indBeforeQuit = 0
    while foundQuitKey < 1:
        if KeysLookedAtFromUserKeys[-nBegin] == 'Quit':
            # look in direction of previous keys
            # if previous key is not quit, you can stop
            if KeysLookedAtFromUserKeys[-nBegin-1] != 'Quit':
                foundQuitKey = 1
                indBeforeQuit = -nBegin - 1
                break
            else:
                nBegin = nBegin + 1
        else:
            # current key is not quit, start looking in the other direction
            if KeysLookedAtFromUserKeys[-nBegin + 1] == 'Quit':
                foundQuitKey = 1
                indBeforeQuit = -nBegin
                break
            else:
                nBegin = nBegin - 1
    
    epochList[-1].insert(2, UserLooksAtKey[indBeforeQuit])
    
    #for i in epochList:
    #    print(i)
    
    return epochList

In [ ]:
# function to convert list of date and time into datetime format list

def timeConversion(timeStrList):
    timeList = list()
    for time in timeStrList:
        time1, t1, t2 = time.partition('+')
        timeList.append(datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
    return timeList

In [ ]:
# This function will return the datetime in items which is the closest to the date pivot
def nearestTimePoint(dates, date):
    
    for d in dates:
        if d < date:
            nearestTP = d
        else:
            continue
    try: 
        nearestTP
        nearestTPind = dates.index(nearestTP)
    except:
        nearestTP = 0
        nearestTPind = -1
        print('no nearest time')
        
    return nearestTP, nearestTPind

In [ ]:
def hampel(vals_orig, k, sd):
    '''
    vals: pandas series of values from which to remove outliers
    k: size of window (including the sample; 7 is equal to 3 on either side of value)
    '''
    # Obtained from: https://stackoverflow.com/questions/46819260/filtering-outliers-how-to-make-median-based-
    # hampel-function-faster
    
    #plt.plot(vals_orig)
    #Make copy so original not edited
    vals1 = pd.DataFrame(vals_orig)      
    vals0 = vals1.replace([np.inf, -np.inf], np.nan)
    vals = vals0.astype(float).interpolate('linear', limit_direction = 'both') # linear interpolation instead of 
    # simply copying the previous value --\ linear interpolation than cubic to not add any patterns in the data, limit direction
    # set to both, to interpolate the nan values occuring from the start of the series
    
    L= 1.4826
    rolling_median = vals.rolling(window=k, min_periods=1, center=True).median()
    
    #print(rolling_median)
    difference = np.abs(rolling_median-vals)
    median_abs_deviation = difference.rolling(k).median()
    threshold = sd * L * median_abs_deviation
    outlier_idx = difference>threshold
    vals[outlier_idx] = rolling_median[outlier_idx]
    #vals.plot()
    return(vals)

In [ ]:
def computeAggregateAverage(trialLookedAtEpoch, GazeLog, pupilData, userKeys, subjName):
    
    nPhrasesTypedEstimate = 0
    userKeyInTrial = list()
    
    # customise for one subj
    if subjName == 'mm_MS':
        trialLookedAtEpoch.remove(trialLookedAtEpoch[-1])
    
    typingMechanism = subjName[-2:]
    rowTimeList = -1
    timeUserLooked = list()
    
    # create a list of times 
    timeStrUserLooked =  [item[0][0] for item in trialLookedAtEpoch]
    timeStrGazeLog = [item3[0] for item3 in GazeLog]
    
    #print(timeStrUserLooked)
    
    # convert the list of strings to datetime formats
    timeUserLooked = timeConversion(timeStrUserLooked)
    timeGazeLog = timeConversion(timeStrGazeLog)
        
    # Create list of pupil sizes from gazelog
    pupilLogL_woFilter = [float(item4[29]) if 'Invalid' not in item4 else np.nan for item4 in GazeLog]
    pupilLogR_woFilter = [float(item5[31]) if 'Invalid' not in item5 else np.nan for item5 in GazeLog]
    
    winSize = 40 # in Per's paper, he used a 13 sample window for 30Hz eye tracker
    # Filter pupil sizes
    pupilLogL = hampel(pupilLogL_woFilter, winSize, 3)
    pupilLogR = hampel(pupilLogR_woFilter, winSize, 3)

    # Pupil distance (x,y,z) in User Coordinate system
    pupilDistLx = [float(item6[3]) if 'Invalid' not in item6 else np.nan for item6 in GazeLog]
    pupilDistLy = [float(item7[4]) if 'Invalid' not in item7 else np.nan for item7 in GazeLog]
    pupilDistLz = [float(item8[5]) if 'Invalid' not in item8 else np.nan for item8 in GazeLog]
    pupilDistRx = [float(item9[10]) if 'Invalid' not in item9 else np.nan for item9 in GazeLog]
    pupilDistRy = [float(item10[11]) if 'Invalid' not in item10 else np.nan for item10 in GazeLog]
    pupilDistRz = [float(item11[12]) if 'Invalid' not in item11 else np.nan for item11 in GazeLog]
    
    # create dataframe from the pupil distances
    df_pupilDist = pd.DataFrame(np.column_stack([pupilDistLx, pupilDistLy, pupilDistLz, pupilDistRx, pupilDistRy, pupilDistRz]), columns=['pupilDistLx', 'pupilDistLy', 'pupilDistLz', 'pupilDistRx', 'pupilDistRy', 'pupilDistRz'])
    df_pupilDist = df_pupilDist.interpolate('linear', limit_direction = 'both')
    #print(df_pupilDist.pupilDistLx)
    
    pupilDist = [math.sqrt((df_pupilDist.pupilDistLx[indPt]-df_pupilDist.pupilDistRx[indPt])**2 + (df_pupilDist.pupilDistLy[indPt]-df_pupilDist.pupilDistRy[indPt])**2 + (df_pupilDist.pupilDistLz[indPt]-df_pupilDist.pupilDistRz[indPt])**2) for indPt in range(0, len(pupilDistLx))]
    
    #print(timeUserLooked)
    
    for timeList in timeUserLooked:
        rowTimeList = rowTimeList + 1
        # Extract epoch data from gazelog
        
        # from the start of look at trial, remove the dwell time passed and fixation time, to get the true starting time
        
        timetrialActive = timeList - datetime.timedelta(milliseconds=float(trialLookedAtEpoch[rowTimeList][0][3])) - datetime.timedelta(milliseconds=TimeFixation)
#        timetrialActive = timeList - datetime.timedelta(milliseconds=float(trialLookedAtEpoch[rowTimeList][3]))

        timeGazeLogStart, GazeLogStartInd = nearestTimePoint(timeGazeLog, timetrialActive)
        
        time1, t1, t2 = trialLookedAtEpoch[rowTimeList][2][0].partition('+')
        timeWindowEnd = (datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
        
        timeGazeLogEnd, GazeLogEndInd = nearestTimePoint(timeGazeLog, timeWindowEnd)
        
        # choose time for reference, which is the time of choosing next phrase
        # for first trial, it needs to be set to 500ms after start, and till now is set to 0
        if trialLookedAtEpoch[rowTimeList][1] == 0:
            # find nearest point in gaze log for end of look at 
            timeReferenceEnd, GazeReferenceEndInd = nearestTimePoint(timeGazeLog, timeList + datetime.timedelta(milliseconds = 500))            
        else:
            # for other trials when the end of next phrase choosing is given
            time1, t1, t2 = trialLookedAtEpoch[rowTimeList][1][0].partition('+')
            timeReference = (datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
            timeReferenceEnd, GazeReferenceEndInd = nearestTimePoint(timeGazeLog, timeReference)
        
        trialReference = [GazeLogStartInd, GazeReferenceEndInd]
            
        
        if timeGazeLogEnd - timeGazeLogStart < datetime.timedelta(seconds = 10):
            #print(timeGazeLogEnd - timeGazeLogStart)
            #print('Too short to be a trial')
            #print(userKeys[userKeys.index(trialTime[rowTimeList][0]):userKeys.index(trialTime[rowTimeList][1])])
            #print('NEXT!!!!!!!!')
            continue
        #else:
        #    print(timeGazeLogEnd - timeGazeLogStart)
            #print('NEXT!!!!!!!!')
        
        
        
        if GazeLogStartInd != GazeLogEndInd:
            pupilWoOutlierL = pupilLogL[GazeLogStartInd:GazeLogEndInd]
            pupilWoOutlierR = pupilLogR[GazeLogStartInd:GazeLogEndInd]
            interPupilDist = pupilDist[GazeLogStartInd:GazeLogEndInd]
            
            winSize = 40
            # Filter pupil sizes
            #pupilWoOutlierL = hampel(gazeEpochL, winSize, 3)
            #pupilWoOutlierR = hampel(gazeEpochR, winSize, 3)
            
            # Does pupilWoOutlier still have nan values -- happens in subject 'lr', the complete list is made of nan values
            if pupilWoOutlierL.isnull().values.any() == True or pupilWoOutlierR.isnull().values.any() == True:
                # This means that the complete list was already only nan values, and cannot be changed, whatsoever
                continue               
            
            # compute correlation between right and left eye pupil sizes
            pupilCorr = pupilWoOutlierL.corrwith(pupilWoOutlierR, axis = 0)
            if pupilCorr.values[0] < 0.8: # 0.8 is a good value for correlation and was also the mean of mean of the correlations
            # for the trials for the users
                #print('correlation less than 0.8')
                continue
             
            nPhrasesTypedEstimate = nPhrasesTypedEstimate + 1
            # Moving Mean of data without outliers: 
            pupilAbsoluteL = pupilWoOutlierL.rolling(window=winSize, min_periods=1, center=True).mean()
            pupilAbsoluteR = pupilWoOutlierR.rolling(window=winSize, min_periods=1, center=True).mean()
            
            #print(subjName, len(pupilWoOutlierL), len(pupilAbsoluteL))
                
            # Reference of when still looking at next phrase for trial 2 onwards or a time of 500ms
            pupilReferenceL = pupilLogL[0][GazeLogStartInd:GazeReferenceEndInd].mean()
            pupilReferenceR = pupilLogR[0][GazeLogStartInd:GazeReferenceEndInd].mean()
            
            #print('pupilAbsoluteL', pupilAbsoluteL)
            #print('ReferenceRange', pupilReferenceL, pupilReferenceR)
            
            pupilRelativeL = (pupilAbsoluteL - pupilReferenceL)/pupilReferenceL
            pupilRelativeR = (pupilAbsoluteR - pupilReferenceR)/pupilReferenceR
            
            #pupilRelativeL = (pupilAbsoluteL - pupilAbsoluteL[0][0])/pupilAbsoluteL[0][0]
            #pupilRelativeR = (pupilAbsoluteR - pupilAbsoluteR[0][0])/pupilAbsoluteR[0][0]
            
            # userKeys looked at
            userKeyInTrial.append(userKeys[userKeys.index(trialLookedAtEpoch[rowTimeList][0]):userKeys.index(trialLookedAtEpoch[rowTimeList][2])])
            #print(len(userKeys[userKeys.index(trialLookedAtEpoch[rowTimeList][0]):userKeys.index(trialLookedAtEpoch[rowTimeList][2])]))
            
            #timeList = [key[0] for key in userKeyInTrial]
            #timeDiff = list()
            #ind = -1
            #for time in timeList:
            #    ind = ind + 1
            #    if ind == 0:
            #        continue
            #    time1, t1, t2 = timeList[ind-1].partition('+')
            #    time2, t3, t4 = time.partition('+')
            #    t = (datetime.datetime.strptime(re.sub('[:.T]','-',time2[:-1]), "%Y-%m-%d-%H-%M-%S-%f") - datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
            #    timeDiff.append(t.total_seconds())
            #print(userKeyInTrial[timeDiff.index(np.max(timeDiff))], userKeyInTrial[timeDiff.index(np.max(timeDiff)) + 1])
            
        else:
            print('one')
            continue
            #continue # remove samples with only 1 sample
        
        #print(pupilRelativeL)
        # Also to check if phrase typed is DT or not, only check the last element of epoch
        #print(timeList[-1])
        if typingMechanism == 'DT':
            # add the pupil size
            if pupilData['DTFirst'] == 0:
                pupilData['DTFirst'] = 1
                
                if absoluteSize:
                    pupilData['DTLeft'] = [pupilAbsoluteL.values[i][0] for i in range(0, len(pupilAbsoluteL.values))]
                    pupilData['DTRight'] = [pupilAbsoluteR.values[i][0] for i in range(0, len(pupilAbsoluteR.values))]
                    
                    pupilData['DTLeftSquared'] = [(pupilAbsoluteL.values[i][0])**2 for i in range(0, len(pupilAbsoluteL.values))]
                    pupilData['DTRightSquared'] = [(pupilAbsoluteR.values[i][0])**2 for i in range(0, len(pupilAbsoluteR.values))]
                    
                else:
                    pupilData['DTLeft'] = [pupilRelativeL.values[i][0] for i in range(0, len(pupilRelativeL.values))]
                    pupilData['DTRight'] = [pupilRelativeR.values[i][0] for i in range(0, len(pupilRelativeR.values))]
                
                    pupilData['DTLeftSquared'] = [(pupilRelativeL.values[i][0])**2 for i in range(0, len(pupilRelativeL.values))]
                    pupilData['DTRightSquared'] = [(pupilRelativeR.values[i][0])**2 for i in range(0, len(pupilRelativeR.values))]
                    
                pupilData['DTNumber'] = pupilData['DTNumber'] + 1
                
                pupilData['DTLeftMax'] = max(pupilData['DTLeft'])
                pupilData['DTRightMax'] = max(pupilData['DTRight'])
                pupilData['DTLeftMin'] = min(pupilData['DTLeft'])
                pupilData['DTRightMin'] = min(pupilData['DTRight'])
                #print(pupilData['DTNumber'])
                
            else:
                if absoluteSize:
                    
                    pupilAddL = [pupilData['DTLeft'][i]+pupilAbsoluteL.values[i][0] for i in range(0, min(len(pupilAbsoluteL[pupilAbsoluteL.columns[0]]), len(pupilData['DTLeft'])))]
                    pupilAddR = [pupilData['DTRight'][i]+pupilAbsoluteR.values[i][0] for i in range(0, min(len(pupilAbsoluteR[pupilAbsoluteR.columns[0]]), len(pupilData['DTRight'])))]
                    
                    pupilAddSquareL = [pupilData['DTLeftSquared'][i]+(pupilAbsoluteL.values[i][0])**2 for i in range(0, min(len(pupilAbsoluteL[pupilAbsoluteL.columns[0]]), len(pupilData['DTLeftSquared'])))]
                    pupilAddSquareR = [pupilData['DTRightSquared'][i]+(pupilAbsoluteR.values[i][0])**2 for i in range(0, min(len(pupilAbsoluteR[pupilAbsoluteR.columns[0]]), len(pupilData['DTRightSquared'])))]
                    
                    pupilData['DTLeftMax'] = max([max(pupilAbsoluteL.values), pupilData['DTLeftMax']])
                    pupilData['DTRightMax'] = max([max(pupilAbsoluteR.values), pupilData['DTRightMax']])
                    pupilData['DTLeftMin'] = min([min(pupilAbsoluteL.values), pupilData['DTLeftMin']])
                    pupilData['DTRightMin'] = min([min(pupilAbsoluteR.values), pupilData['DTRightMin']])
                
                else:
                    pupilAddL = [pupilData['DTLeft'][i]+pupilRelativeL.values[i][0] for i in range(0, min(len(pupilRelativeL[pupilRelativeL.columns[0]]), len(pupilData['DTLeft'])))]
                    pupilAddR = [pupilData['DTRight'][i]+pupilRelativeR.values[i][0] for i in range(0, min(len(pupilRelativeR[pupilRelativeR.columns[0]]), len(pupilData['DTRight'])))]
                    
                    pupilAddSquareL = [pupilData['DTLeftSquared'][i]+(pupilRelativeL.values[i][0])**2 for i in range(0, min(len(pupilRelativeL[pupilRelativeL.columns[0]]), len(pupilData['DTLeftSquared'])))]
                    pupilAddSquareR = [pupilData['DTRightSquared'][i]+(pupilRelativeR.values[i][0])**2 for i in range(0, min(len(pupilRelativeR[pupilRelativeR.columns[0]]), len(pupilData['DTRightSquared'])))]
                    
                    pupilData['DTLeftMax'] = max([max(pupilRelativeL.values), pupilData['DTLeftMax']])
                    pupilData['DTRightMax'] = max([max(pupilRelativeR.values), pupilData['DTRightMax']])
                    pupilData['DTLeftMin'] = min([min(pupilRelativeL.values), pupilData['DTLeftMin']])
                    pupilData['DTRightMin'] = min([min(pupilRelativeR.values), pupilData['DTRightMin']])
                
                pupilData['DTLeft'] = pupilAddL
                pupilData['DTRight'] = pupilAddR
                pupilData['DTLeftSquared'] = pupilAddSquareL
                pupilData['DTRightSquared'] = pupilAddSquareR
                pupilData['DTNumber'] = pupilData['DTNumber'] + 1
                
                
                
        else:
            if pupilData['MSFirst'] == 0:
                pupilData['MSFirst'] = 1
                if absoluteSize:
                    
                    pupilData['MSLeft'] = [pupilAbsoluteL.values[i][0] for i in range(0, len(pupilAbsoluteL.values))]
                    pupilData['MSRight'] = [pupilAbsoluteR.values[i][0] for i in range(0, len(pupilAbsoluteR.values))]
                    
                    pupilData['MSLeftSquared'] = [(pupilAbsoluteL.values[i][0])**2 for i in range(0, len(pupilAbsoluteL.values))]
                    pupilData['MSRightSquared'] = [(pupilAbsoluteR.values[i][0])**2 for i in range(0, len(pupilAbsoluteR.values))]
                    
                else:
                    pupilData['MSLeft'] = [pupilRelativeL.values[i][0] for i in range(0, len(pupilRelativeL.values))]
                    pupilData['MSRight'] = [pupilRelativeR.values[i][0] for i in range(0, len(pupilRelativeR.values))]
                    
                    pupilData['MSLeftSquared'] = [(pupilRelativeL.values[i][0])**2 for i in range(0, len(pupilRelativeL.values))]
                    pupilData['MSRightSquared'] = [(pupilRelativeR.values[i][0])**2 for i in range(0, len(pupilRelativeR.values))]
                    
                
                pupilData['MSNumber'] = pupilData['MSNumber'] + 1
                
                pupilData['MSLeftMax'] = max(pupilData['MSLeft'])
                pupilData['MSRightMax'] = max(pupilData['MSRight'])
                pupilData['MSLeftMin'] = min(pupilData['MSLeft'])
                pupilData['MSRightMin'] = min(pupilData['MSRight'])
                
                #print(subjName, 'MS')

            else:
                if absoluteSize:
                    pupilAddL = [pupilData['MSLeft'][i]+pupilAbsoluteL.values[i][0] for i in range(0, min(len(pupilAbsoluteL[pupilAbsoluteL.columns[0]]), len(pupilData['MSLeft'])))]
                    pupilAddR = [pupilData['MSRight'][i]+pupilAbsoluteR.values[i][0] for i in range(0, min(len(pupilAbsoluteR[pupilAbsoluteR.columns[0]]), len(pupilData['MSRight'])))]
                
                    pupilAddSquareL = [pupilData['MSLeft'][i]+(pupilAbsoluteL.values[i][0])**2 for i in range(0, min(len(pupilAbsoluteL[pupilAbsoluteL.columns[0]]), len(pupilData['MSLeftSquared'])))]
                    pupilAddSquareR = [pupilData['MSRight'][i]+(pupilAbsoluteR.values[i][0])**2 for i in range(0, min(len(pupilAbsoluteR[pupilAbsoluteR.columns[0]]), len(pupilData['MSRightSquared'])))]
                    
                    pupilData['MSLeftMax'] = max([max(pupilAbsoluteL.values), pupilData['MSLeftMax']])
                    pupilData['MSRightMax'] = max([max(pupilAbsoluteR.values), pupilData['MSRightMax']])
                    pupilData['MSLeftMin'] = min([min(pupilAbsoluteL.values), pupilData['MSLeftMin']])
                    pupilData['MSRightMin'] = min([min(pupilAbsoluteR.values), pupilData['MSRightMin']])
                    
                else:
                    pupilAddL = [pupilData['MSLeft'][i]+pupilRelativeL.values[i][0] for i in range(0, min(len(pupilRelativeL[pupilRelativeL.columns[0]]), len(pupilData['MSLeft'])))]
                    pupilAddR = [pupilData['MSRight'][i]+pupilRelativeR.values[i][0] for i in range(0, min(len(pupilRelativeR[pupilRelativeR.columns[0]]), len(pupilData['MSRight'])))]
                    
                    pupilAddSquareL = [pupilData['MSLeftSquared'][i]+(pupilRelativeL.values[i][0])**2 for i in range(0, min(len(pupilRelativeL[pupilRelativeL.columns[0]]), len(pupilData['MSLeft'])))]
                    pupilAddSquareR = [pupilData['MSRightSquared'][i]+(pupilRelativeR.values[i][0])**2 for i in range(0, min(len(pupilRelativeR[pupilRelativeR.columns[0]]), len(pupilData['MSRight'])))]
                    
                    #print('Maximum', max(pupilRelativeL.values), pupilData['MSLeftMax'])
                    pupilData['MSLeftMax'] = max([max(pupilRelativeL.values), pupilData['MSLeftMax']])
                    pupilData['MSRightMax'] = max([max(pupilRelativeR.values), pupilData['MSRightMax']])
                    pupilData['MSLeftMin'] = min([min(pupilRelativeL.values), pupilData['MSLeftMin']])
                    pupilData['MSRightMin'] = min([min(pupilRelativeR.values), pupilData['MSRightMin']])                    
                    
                pupilData['MSLeft'] = pupilAddL
                pupilData['MSRight'] = pupilAddR
                pupilData['MSLeftSquared'] = pupilAddSquareL
                pupilData['MSRightSquared'] = pupilAddSquareR
                pupilData['MSNumber'] = pupilData['MSNumber'] + 1
                
    #print(len(userKeyInTrial))
    #print('Number of phrases typed can be estimated to be ', nPhrasesTypedEstimate)       
    return pupilData, userKeyInTrial

In [ ]:
keyLocation = dict()

keyLocation['scratchPadAndSuggestions'] = ['ScratchPad', 'NextSuggestions', 'Suggestion1', 'Suggestion2', 'Suggestion3', 'Suggestion4', 
                             'Suggestion5', 'PreviousSuggestions']
keyLocation['phraseText'] = ['PhraseTextBlock']
#keyLocation['sleepKey'] = ['Sleep']
keyLocation['nextPhraseOrQuitOrSleep'] = ['NextPhrase', 'Quit', 'YesQuestionResult', 'NoQuestionResult', 'Sleep']
keyLocation['characterKey'] = ['q', 'w', 'e', 'r', 't', 'y', 'u', 'i', 'o', 'p', 'å', 'a', 's', 'd', 'f', 'g', 'h', 'j', 'k',
                               'l', 'æ', 'ø', 'z', 'x', 'c', 'v', 'b', 'n', 'm', '.', ',', 'SpaceBar', 'LeftShift', 'Comma', 'MultiKeySelectionIsOn']
keyLocation['dwellTime'] = ['IncreaseDwellTime', 'DecreaseDwellTime']

In [ ]:
def AggregateUserLookLocation(userKeysInTrialList, categoryList, keyLocation, subjName):
    
    typingMechanism = subjName[-2:]
    # go through the list and divide into categories of scratchPad + suggestions, character keys, sleep, phraseText or Quit 
    # and others
    
    if typingMechanism == 'DT':
        for list5s in userKeysInTrialList:
            keyInd = -1
            
            for key in list5s:
                keyInd = keyInd + 1
            
                if len(categoryList['DT']) > keyInd:
                    # this list exists and the values need only be modified
                    if key[1] in keyLocation['scratchPadAndSuggestions']:
                        categoryList['DT'][keyInd][0] = categoryList['DT'][keyInd][0] + 1
                        #print(key)
                    elif key[1] in keyLocation['phraseText']:
                        categoryList['DT'][keyInd][1] = categoryList['DT'][keyInd][1] + 1
                    elif key[1] in keyLocation['nextPhraseOrQuitOrSleep']:
                        categoryList['DT'][keyInd][2] = categoryList['DT'][keyInd][2] + 1
                    elif key[1] in keyLocation['characterKey']:
                        categoryList['DT'][keyInd][3] = categoryList['DT'][keyInd][3] + 1
                    else: 
                        # others
                        #print(key)
                        categoryList['DT'][keyInd][4] = categoryList['DT'][keyInd][4] + 1
              
                else:
                    # add a list of 6 elements to the list element
                    categoryList['DT'].append([0, 0, 0, 0, 0])
                    # list does not exist and values need be appended
                    if key[1] in keyLocation['scratchPadAndSuggestions']:
                        categoryList['DT'][keyInd][0] = categoryList['DT'][keyInd][0] + 1
                    elif key[1] in keyLocation['phraseText']:
                        categoryList['DT'][keyInd][1] = categoryList['DT'][keyInd][1] + 1
                    elif key[1] in keyLocation['nextPhraseOrQuitOrSleep']:
                        categoryList['DT'][keyInd][2] = categoryList['DT'][keyInd][2] + 1
                    elif key[1] in keyLocation['characterKey']:
                        categoryList['DT'][keyInd][3] = categoryList['DT'][keyInd][3] + 1
                    else: 
                        # others
                        #print(key)
                        categoryList['DT'][keyInd][4] = categoryList['DT'][keyInd][4] + 1
        
    else:
        
        for list5s in userKeysInTrialList:
            keyInd = -1

            for key in list5s:
                keyInd = keyInd + 1
            
                if len(categoryList['MS']) > keyInd:
                    # this list exists and the values need only be modified
                    if key[1] in keyLocation['scratchPadAndSuggestions']:
                        categoryList['MS'][keyInd][0] = categoryList['MS'][keyInd][0] + 1
                        #print(key)
                    elif key[1] in keyLocation['phraseText']:
                        categoryList['MS'][keyInd][1] = categoryList['MS'][keyInd][1] + 1
                    elif key[1] in keyLocation['nextPhraseOrQuitOrSleep']:
                        categoryList['MS'][keyInd][2] = categoryList['MS'][keyInd][2] + 1
                    elif key[1] in keyLocation['characterKey']:
                        categoryList['MS'][keyInd][3] = categoryList['MS'][keyInd][3] + 1
                    else: 
                        # others
                        #print(key)
                        categoryList['MS'][keyInd][4] = categoryList['MS'][keyInd][4] + 1
                    
              
                else:
                    # add a list of 6 elements to the list element
                    categoryList['MS'].append([0, 0, 0, 0, 0])
                    # list does not exist and values need be appended
                    
                    if key[1] in keyLocation['scratchPadAndSuggestions']:
                        categoryList['MS'][keyInd][0] = categoryList['MS'][keyInd][0] + 1
                    elif key[1] in keyLocation['phraseText']:
                        categoryList['MS'][keyInd][1] = categoryList['MS'][keyInd][1] + 1
                    elif key[1] in keyLocation['nextPhraseOrQuitOrSleep']:
                        categoryList['MS'][keyInd][2] = categoryList['MS'][keyInd][2] + 1
                    elif key[1] in keyLocation['characterKey']:
                        categoryList['MS'][keyInd][3] = categoryList['MS'][keyInd][3] + 1
                    else: 
                        # others
                        #print(key)
                        categoryList['MS'][keyInd][4] = categoryList['MS'][keyInd][4] + 1
     
        
    return categoryList

In [ ]:
subjName = r'C:\DTU\Data\201805_HealthnRehab\TypingData'
j = 0
flagFirstSubj = 0
pupilData = dict()
pupilData['DTFirst'] = 0
pupilData['MSFirst'] = 0
pupilData['DTNumber'] = 0
pupilData['MSNumber'] = 0

categoryList = dict()
categoryList['DT'] = list()
categoryList['MS'] = list()


for root, dirs, subfolder in os.walk(subjName):
    LetterLookedAtList = list()
    LetterLookedAt = list()
    
    if not dirs:
        
        if 'notCompleted' in root or 'notInclude' in root: # Some subjects do not have gaze log and have been marked as 
            #notInclude
            continue
        if 'tb' in root or 'joha' in root:
            continue
            
        userKeys = None
        scratchPad = None
        gazeLog = None
        stimPhrase = None
        
        for file in subfolder:
            
            if fnmatch.fnmatch(file, 'user_look*'):
                try:
                    
                    fUserKey = open(root + '\\' + file, encoding='utf-8')
                    readerUserKey = csv.reader(fUserKey)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user looks at log file')
            elif fnmatch.fnmatch(file, 'ScratchPad*'):
                try:
                    fScratchPad = open(root + '\\' + file, encoding='utf-8')
                    readerScratchPad = csv.reader(fScratchPad)
                    scratchPad = list(readerScratchPad)  
                    scratchPad.remove(scratchPad[0])
                except:
                    if fScratchPad is not None:
                        fScratchPad.close()
                    else:
                        print('error in opening the user looks at log file')
            elif fnmatch.fnmatch(file, 'PhraseLog*'):
                try:
                    fStimPhrase = open(root + '\\' + file, encoding='utf-8')
                    readerStimPhrase = csv.reader(fStimPhrase)
                    stimPhrase = list(readerStimPhrase)
                    stimPhrase.remove(stimPhrase[0])
                except:
                    if fStimPhrase is not None:
                        fStimPhrase.close()
                    else:
                        print('error in opening the phrase log file')
            elif fnmatch.fnmatch(file, 'GazeLog*'):
                try:
                    fGazeLog = open(root + '\\' + file, encoding='utf-8')
                    readerGazeLog = csv.reader(fGazeLog)
                    gazeLog = list(readerGazeLog)
                    gazeLog.remove(gazeLog[0]) # would not matter much even if the first row was not labels
                    gazeLog.remove(gazeLog[-1])
                except:
                    if fGazeLog is not None:
                        fGazeLog.close()
                    else:
                        print('error in opening the gaze log file')
            else:
                continue
            
                # if all these lists exist
            if userKeys is None or scratchPad is None or stimPhrase is None or gazeLog is None:
                continue
            else:

                # Compute dwell time
                userKeysWithTime = ComputeDwellTime(userKeys)

                # call function to check when scratchpad is looked at and save it in a list
                trialTime = UserLookedAtNewTrial(userKeysWithTime)
                
                # for every element, find the time closest and previous to it, and check what was typed 
                # AND what should have been typed
                # Also, add gaze data to epoch, but first subtract the fixation time and complete the pupil data filtering as
                # per Per's paper
                
                a = re.compile('(?<=TypingData\\\May[0-9]{2}\\\)(.*)(?=\\\OptiKey)')
                subjName = a.findall(root)[0]
                print(subjName)
                pupilData, userKeysInTrial = computeAggregateAverage(trialTime, gazeLog, pupilData, userKeysWithTime, subjName)
                if userKeysInTrial is not None:
                    categoryList = AggregateUserLookLocation(userKeysInTrial, categoryList, keyLocation, subjName)


In [ ]:
if pupilData['DTFirst'] > 0 and pupilData['MSFirst'] > 0:
    #print(pupilData)
    # if the DT and MS data are of different sizes,
    pupilSizeMin = min(len(pupilData['DTLeft']), len(pupilData['MSLeft']))
    fig = plt.figure()
    axL = fig.add_subplot(2, 2, 1)
    axR = fig.add_subplot(2, 2, 2)
    xAxis = np.arange(0, float(pupilSizeMin/90), float(1/90))
    pupilPlot = dict()
    pupilPlot['DTLeft'] = [x/pupilData['DTNumber'] for x in pupilData['DTLeft']]
    pupilPlot['DTRight'] = [x/pupilData['DTNumber'] for x in pupilData['DTRight']]
    pupilPlot['MSLeft'] = [x/pupilData['MSNumber'] for x in pupilData['MSLeft']]
    pupilPlot['MSRight'] = [x/pupilData['MSNumber'] for x in pupilData['MSRight']]
    
    # Standard deviation
#     pupilPlot['DTLeftSquareRoot'] = [(math.fabs(pupilData['DTLeftSquared'][xInd]/pupilData['DTNumber']-(pupilPlot['DTLeft'][xInd])**2))**0.5 for xInd in range(0, len(pupilData['DTLeftSquared']))]
#     pupilPlot['DTRightSquareRoot'] = [(math.fabs(pupilData['DTRightSquared'][xInd]/pupilData['DTNumber']-(pupilPlot['DTRight'][xInd]))**2)**0.5 for xInd in range(0, len(pupilData['DTRightSquared']))]
#     pupilPlot['MSLeftSquareRoot'] = [(math.fabs(pupilData['MSLeftSquared'][xInd]/pupilData['MSNumber']-(pupilPlot['MSLeft'][xInd])**2))**0.5 for xInd in range(0, len(pupilData['MSLeftSquared']))]
#     pupilPlot['MSRightSquareRoot'] = [(math.fabs(pupilData['MSRightSquared'][xInd]/pupilData['MSNumber']-(pupilPlot['MSRight'][xInd])**2))**0.5 for xInd in range(0, len(pupilData['MSRightSquared']))]

    pupilPlot['DTLeftSquareRoot'] = [(math.fabs(((pupilData['DTNumber'])*(pupilData['DTLeftSquared'][xInd])-(pupilData['DTLeft'][xInd])**2)/((pupilData['DTNumber'])*(pupilData['DTNumber']-1))))**0.5 for xInd in range(0, len(pupilData['DTLeftSquared']))]
    pupilPlot['DTRightSquareRoot'] = [(math.fabs(((pupilData['DTNumber'])*(pupilData['DTRightSquared'][xInd])-(pupilData['DTRight'][xInd])**2)/((pupilData['DTNumber'])*(pupilData['DTNumber']-1))))**0.5 for xInd in range(0, len(pupilData['DTRightSquared']))]
    pupilPlot['MSLeftSquareRoot'] = [(math.fabs(((pupilData['MSNumber'])*(pupilData['MSLeftSquared'][xInd])-(pupilData['MSLeft'][xInd])**2)/((pupilData['MSNumber'])*(pupilData['MSNumber']-1))))**0.5 for xInd in range(0, len(pupilData['MSLeftSquared']))]
    pupilPlot['MSRightSquareRoot'] = [(math.fabs(((pupilData['MSNumber'])*(pupilData['MSRightSquared'][xInd])-(pupilData['MSRight'][xInd])**2)/((pupilData['MSNumber'])*(pupilData['MSNumber']-1))))**0.5 for xInd in range(0, len(pupilData['MSRightSquared']))]

    pupilPlot['DTLeftStdPositive'] = [pupilPlot['DTLeft'][xInd] + pupilPlot['DTLeftSquareRoot'][xInd] for xInd in range(0, len(pupilPlot['DTLeft']))]
    pupilPlot['DTLeftStdNegative'] = [pupilPlot['DTLeft'][xInd] - pupilPlot['DTLeftSquareRoot'][xInd] for xInd in range(0, len(pupilPlot['DTLeft']))]
    pupilPlot['MSLeftStdPositive'] = [pupilPlot['MSLeft'][xInd] + pupilPlot['MSLeftSquareRoot'][xInd] for xInd in range(0, len(pupilPlot['MSLeft']))]
    pupilPlot['MSLeftStdNegative'] = [pupilPlot['MSLeft'][xInd] - pupilPlot['MSLeftSquareRoot'][xInd] for xInd in range(0, len(pupilPlot['MSLeft']))]

    pupilPlot['DTRightStdPositive'] = [pupilPlot['DTRight'][xInd] + pupilPlot['DTRightSquareRoot'][xInd] for xInd in range(0, len(pupilPlot['DTRight']))]
    pupilPlot['DTRightStdNegative'] = [pupilPlot['DTRight'][xInd] - pupilPlot['DTRightSquareRoot'][xInd] for xInd in range(0, len(pupilPlot['DTRight']))]
    pupilPlot['MSRightStdPositive'] = [pupilPlot['MSRight'][xInd] + pupilPlot['MSRightSquareRoot'][xInd] for xInd in range(0, len(pupilPlot['MSRight']))]
    pupilPlot['MSRightStdNegative'] = [pupilPlot['MSRight'][xInd] - pupilPlot['MSRightSquareRoot'][xInd] for xInd in range(0, len(pupilPlot['MSRight']))]


    axL.plot(xAxis, pupilPlot['DTLeft'][0:pupilSizeMin], 'bo', label = 'Dwell Time')
    axL.plot(xAxis, pupilPlot['MSLeft'][0:pupilSizeMin], 'ro', label = 'Multi-key Selection')
    axL.fill_between(xAxis, pupilPlot['DTLeftStdPositive'][0:pupilSizeMin],  pupilPlot['DTLeftStdNegative'][0:pupilSizeMin], alpha = 0.15, color = 'blue')
    axL.fill_between(xAxis, pupilPlot['MSLeftStdPositive'][0:pupilSizeMin], pupilPlot['MSLeftStdNegative'][0:pupilSizeMin], alpha = 0.15, color = 'red')

    
    axL.set_title('Left pupil')
    axL.set_xlabel('Time [in s]')
    if absoluteSize:
        axL.set_ylabel('Absolute pupil size [in mm]')
    else:
        axL.set_ylabel('Relative pupil size')
    axL.legend()

    axR.plot(xAxis, pupilPlot['DTRight'][0:pupilSizeMin], 'bo', label = 'Dwell Time')
    axR.plot(xAxis, pupilPlot['MSRight'][0:pupilSizeMin], 'ro', label = 'Multi-key Selection')
    axR.fill_between(xAxis, pupilPlot['DTRightStdPositive'][0:pupilSizeMin], pupilPlot['DTRightStdNegative'][0:pupilSizeMin], alpha = 0.15, color = 'blue')
    axR.fill_between(xAxis, pupilPlot['MSRightStdPositive'][0:pupilSizeMin], pupilPlot['MSRightStdNegative'][0:pupilSizeMin], alpha = 0.15, color = 'red')
    axR.set_title('Right pupil')
    axR.legend()
    axR.set_xlabel('Time [in s]')
    if absoluteSize:
        axR.set_ylabel('Absolute pupil size [in mm]')
    else:
        axR.set_ylabel('Relative pupil size')
    

In [ ]:
# make both lists equal

print(len(categoryList['DT']), len(categoryList['MS']))

#if (len(categoryList['DT']) - len(categoryList['MS'])) > 0:
    
#    for i in range(0, len(categoryList['DT']) - len(categoryList['MS'])):
#        categoryList['MS'].append([0]*5)

In [ ]:
# plot the category list
normaliseFreq = True

categoryListDT = categoryList['DT']
# normalise the category list
if normaliseFreq:
    print(normaliseFreq)
    rowInd = -1
    for row in categoryListDT:
        rowInd = rowInd + 1
        sumOfRow = sum(row)
        categoryInd = -1
        for category in row:
            categoryInd = categoryInd + 1
            categoryListDT[rowInd][categoryInd] = categoryListDT[rowInd][categoryInd]/sumOfRow


categoryListMS = categoryList['MS']
# normalise the category list
if normaliseFreq:
    rowInd = -1
    for row in categoryListMS:
        rowInd = rowInd + 1
        sumOfRow = sum(row)
        if sumOfRow == 0:
            sumOfRow = 1
        categoryInd = -1
        for category in row:
            categoryInd = categoryInd + 1
            categoryListMS[rowInd][categoryInd] = categoryListMS[rowInd][categoryInd]/sumOfRow

# convert category list to individual lists
scratchPadAndSuggestionsListDT = [key[0] for key in categoryListDT]
phraseTextListDT = [key[1] for key in categoryListDT]
#sleepListDT = [key[2] for key in categoryListDT]
nextPhraseOrQuitOrSleepListDT = [key[2] for key in categoryListDT]
characterListDT = [key[3] for key in categoryListDT]
dwellTimeListDT = [key[4] for key in categoryListDT]

# convert category list to individual lists
scratchPadAndSuggestionsListMS = [key[0] for key in categoryListMS]
phraseTextListMS = [key[1] for key in categoryListMS]
#sleepListMS = [key[2] for key in categoryListMS]
nextPhraseOrQuitOrSleepListMS = [key[2] for key in categoryListMS]
characterListMS = [key[3] for key in categoryListMS]
dwellTimeListMS = [key[4] for key in categoryListMS]


bottomForNextPhraseDT = [scratchPadAndSuggestionsListDT[i] + characterListDT[i] for i in range(0, len(characterListDT))]
bottomForPhraseTextDT = [scratchPadAndSuggestionsListDT[i] + characterListDT[i] + nextPhraseOrQuitOrSleepListDT[i] for i in range(0, len(characterListDT))]
bottomForDwellTimeDT = [scratchPadAndSuggestionsListDT[i] + characterListDT[i] + nextPhraseOrQuitOrSleepListDT[i] + phraseTextListDT[i] for i in range(0, len(characterListDT))]


bottomForNextPhraseMS = [scratchPadAndSuggestionsListMS[i] + characterListMS[i] for i in range(0, len(characterListMS))]
bottomForPhraseTextMS = [scratchPadAndSuggestionsListMS[i] + characterListMS[i] + nextPhraseOrQuitOrSleepListMS[i] for i in range(0, len(characterListMS))]
bottomForDwellTimeMS = [scratchPadAndSuggestionsListMS[i] + characterListMS[i] + nextPhraseOrQuitOrSleepListMS[i] + phraseTextListMS[i] for i in range(0, len(characterListMS))]





In [ ]:
categoryList['MS'][0:5]

In [ ]:
xaxisDT = np.arange(0, len(categoryList['DT']), 1)
xaxis_ticksDT = np.arange(0, 12, 0.00036)


ax3 = fig.add_subplot(2,2,3)
ax3.bar(xaxisDT, scratchPadAndSuggestionsListDT)
ax3.bar(xaxisDT, characterListDT, bottom = scratchPadAndSuggestionsListDT)
#ax3.bar(xaxisDT, sleepListDT, label = 'sleep key')
ax3.bar(xaxisDT, nextPhraseOrQuitOrSleepListDT, bottom = bottomForNextPhraseDT)
ax3.bar(xaxisDT, phraseTextListDT, bottom = bottomForPhraseTextDT)
ax3.bar(xaxisDT, dwellTimeListDT, bottom = bottomForDwellTimeDT, label = 'dwell time changed')
#ax3.legend( loc = 'upper right')


ax3.set_xlim([0, len(categoryList['DT'])])
#ax3.set_xticklabels([round(xaxis_ticksDT[0], 2), round(xaxis_ticksDT[50], 2) , round(xaxis_ticksDT[100], 2) , round(xaxis_ticksDT[150], 2) , round(xaxis_ticksDT[200], 2) , round(xaxis_ticksDT[250], 2), round(xaxis_ticksDT[300], 2), round(xaxis_ticksDT[-1], 2)])
ax3.set_title('Dwell time')
ax3.set_xlabel('Time [in s]')
ax3.set_ylabel('Cumulative Frequency')
xaxisMS = np.arange(0, len(categoryList['MS']), 1)
xaxis_ticksMS = np.arange(0, 5.15, 0.015)
tickLabel_DT = ax3.get_xticks()
#locs_DT = ax3.get_xticklabels()
#ax3_xtickLabels = locs_DT*0.03
#ax3.set_xticklabels(ax3_xtickLabels)


ax4 = fig.add_subplot(2,2,4)


ax4.bar(xaxisMS, scratchPadAndSuggestionsListMS, label = 'scratchpad and suggestions')
ax4.bar(xaxisMS, characterListMS, bottom = scratchPadAndSuggestionsListMS, label = 'character keys')
#ax4.bar(xaxisMS, sleepListMS, label = 'sleep key')
ax4.bar(xaxisMS, nextPhraseOrQuitOrSleepListMS, bottom = bottomForNextPhraseMS, label = 'next phrase or quit or sleep')
ax4.bar(xaxisMS, phraseTextListMS, bottom = bottomForPhraseTextMS, label = 'phrase text')
ax4.bar(xaxisMS, dwellTimeListMS, bottom = bottomForDwellTimeMS, label = 'dwell time changed')
ax4.legend()

ax4.set_xlim([0, len(categoryList['DT'])])
#ax4.set_xticklabels([round(xaxis_ticksDT[0], 2), round(xaxis_ticksDT[50], 2) , round(xaxis_ticksDT[100], 2) , round(xaxis_ticksDT[150], 2) , round(xaxis_ticksDT[200], 2) , round(xaxis_ticksDT[250], 2), round(xaxis_ticksDT[300], 2), round(xaxis_ticksDT[-1], 2)])
ax4.set_title('Multi-key Selection')
ax4.set_xlabel('Time [in s]')
ax4.set_ylabel('Cumulative Frequency')
tickLabel_MS = ax4.get_xticks()
#locs_MS = ax4.get_xticklabels()
#ax4_xtickLabels = locs_MS*0.03
#ax4.set_xticklabels(ax4_xtickLabels)


In [ ]:
for i in locs_DT:
    print(i)

In [ ]:
tickLabel_DT

In [ ]:
a = [[1, 2, 2], [2, 3]]
a0 = [b[0] for b in a]

In [ ]:
a0 = list()
ind = -1
for b in a:
    ind  = ind + 1
    if len(b) > ind:
        print(b[ind])
        a0.append(b[ind])
    else:
        continue

In [ ]:
pupilData['DTNumber']

In [ ]:
pupilData['MSNumber']